In [16]:
%load_ext autoreload
%autoreload 2
from math import log
from pathlib import Path
import os
import subprocess
import json
import sys
import time
import numpy as np
import vtk
import shutil
from pymskt.mesh import get_icp_transform, cpd_register, non_rigidly_register
from pymskt import mesh
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy
from pymskt.mesh.meshTools import smooth_scalars_from_second_mesh_onto_base, transfer_mesh_scalars_get_weighted_average_n_closest
from pymskt.mesh import Mesh, BoneMesh
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pickle

In [ ]:
# Constants and configurations
knee_to_use = ['aclr', 'clat', 'ctrl']
dir_path = '/dataNAS/people/anoopai//DESS_ACL_study'
code_dir_path = '/dataNAS/people/anoopai/KneePipeline/'
data_path = os.path.join(dir_path, 'data')
log_path = '/dataNAS/people/anoopai/KneePipeline/logs'
output_file = os.path.join(log_path, f't2_and_thickness_change.txt')
log_file_path = os.path.join(log_path, f'pipeline_DESS_errors.txt')
mean_path = os.path.join(code_dir_path, 'mean_data')
save_path = os.path.join(mean_path, f't2_and_thickness_change')
mean_femur_path = os.path.join(mean_path, 'MeanFemur_mesh_B-only.vtk')

parameter1= 'Diff_thickness (mm)'
parameter2= 'Diff_T2_mean_filt'

# parameter1= 'thickness (mm)'
# parameter2= 'T2_mean'

In [ ]:
# List to keep track of the dictionaries
analysis_complete = []
data_all = []

for subject in os.listdir(data_path):
    subject_path = os.path.join(data_path, subject)

    if os.path.isdir(subject_path):
        # Iterate through each visit in the subject path
        for visit in os.listdir(subject_path):
            if visit not in ['VISIT-6', 'VISIT-1']:  # Exclude VISIT-6
                visit_path = os.path.join(subject_path, visit)

                if os.path.isdir(visit_path):
                    # Iterate through each knee type in the visit path
                    for knee in os.listdir(visit_path):
                        knee_path = os.path.join(visit_path, knee)
                        
                        if os.path.isdir(knee_path) and knee in knee_to_use:
                            path_image = os.path.join(knee_path, 'scans/qdess')
                            path_save = os.path.join(knee_path, f'results_nsm')
                            femur_path = os.path.join(path_save, 'femur_mesh_NSM_orig_reg2mean.vtk')
                            
                            sub_component= Path(knee_path).parts[6]  # '11-P'
                            visit_component = Path(knee_path).parts[7]  # 'VISIT-1'
                            knee_component = Path(knee_path).parts[8]  # 'clat'
                            try:
                            
                                if os.path.exists(femur_path):
                                    femur_mesh = BoneMesh(femur_path)
                                    thickness_change_all = {}
                                    t2_change_all = {}
                                    scalar_names =[parameter1, parameter2]
                                    data= {'sub': sub_component,
                                            'visit': visit_component,
                                            'knee': knee_component
                                            }

                                    for scalar_name in scalar_names:
                                        data[scalar_name] = vtk_to_numpy(femur_mesh.GetPointData().GetArray(scalar_name))
                                    
                                    data_all.append(data)
                                else:
                                    print(f"Femur mesh not found for {subject} {visit} {knee}")
                                    continue    
                            except Exception as e:
                                print(f"Error processing {subject} {visit} {knee}: {e}")
                                continue

In [ ]:
# visits = ['VISIT-1', 'VISIT-2', 'VISIT-3', 'VISIT-4', 'VISIT-5']
visits = ['VISIT-2', 'VISIT-3', 'VISIT-4', 'VISIT-5']
knees = ['aclr', 'clat', 'ctrl']

# Create a dictionary to store filtered results
filtered_results = {knee: {visit: [] for visit in visits} for knee in knees}

# Filter data
for d in data_all:
    knee_type = d['knee']
    visit_type = d['visit']
    if knee_type in knees and visit_type in visits:
        filtered_results[knee_type][visit_type].append(d)

# Access filtered results
aclr_results = filtered_results['aclr']
clat_results = filtered_results['clat']
ctrl_results = filtered_results['ctrl']

# Print results
print("ACLR Results:")
print(aclr_results)
print("\nCLAT Results:")
print(clat_results)
print("\nCTRL Results:")
print(ctrl_results)


In [ ]:
ctrl_v2= ctrl_results['VISIT-2']

ctrl_v2_t2c = []
for entry in ctrl_v2:
    # print(entry)
    row = (entry['Diff_T2_mean_filt'])
    ctrl_v2_t2c.append(row)

# Convert to numpy array
ctrl_v2_t2c = np.array(ctrl_v2_t2c)
ctrl_v2_t2c_fc_only = ctrl_v2_t2c[(ctrl_v2_t2c != 100) & (ctrl_v2_t2c != 200)]


# # Save to npy file
np.save('ctrl_v2_diffmaps.npy', ctrl_v2_t2c_fc_only)

# ctrl_v2_t2c.flatten()
plt.hist(ctrl_v2_t2c_fc_only.flatten(), bins=100)


In [ ]:
def compute_intensity_threshold(difference_maps_all, std_values):
    
    '''
    This function computes the intensity threshold for the T2-Cluster analysis.
    
    Parameters:
    
    difference_maps_all (array): A 4D NumPy array containing the 3D difference maps (axes 0-2) for all subjects (axis 3).
    std_vales = A list of standard deviation values to calculate the intensity threshold at
    
    '''
    

    import os
    import numpy as np
    import nibabel as nib
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    # from utils.append_df_to_excel import append_df_to_excel

    # Initialize an empty list to store the non-NaN values from all images
    non_nan_values_all = np.array([])
    
    # Initialize an empty list to store the non-NaN values from all images
    non_nan_values = []
    
    # Flatten the 3D array and remove NaN values
    non_nan_values.extend(difference_maps_all[~np.isnan(difference_maps_all)])

    # Convert the list of non-NaN values to a NumPy array
    non_nan_values_all = np.array(non_nan_values)

    mean_value = np.mean(non_nan_values)
    std_dev = np.std(non_nan_values)
    
    intensity_threshold_data = pd.DataFrame({
            'Mean': [mean_value],
        })

    # Calculate the value at x standard deviations
    for std_value in std_values:
        intensity_threshold = mean_value + std_value * std_dev
        intensity_threshold_data[f'{std_value}*Std'] = mean_value + std_value * std_dev
        
    print(intensity_threshold_data)
        
    fig, ax= plt.subplots(figsize=(5, 5))
    sns.histplot(non_nan_values_all,  
                # kde=True,
                bins='auto', 
                color = '#B8860B', 
                # hist_kws={'edgecolor':'#B8860B'},
                # kde_kws={'linewidth': 1},
                stat = 'density',
                ax=ax,
                )
    ax.set_xlim(-20, 20)
    
    for std_value in std_values:

        # Calculate the value at 2 standard deviations
        stdev = mean_value + std_value * std_dev

        # Draw a vertical line at 2 standard deviations above the mean
        plt.axvline(x=stdev, color='#8B3E2F', linestyle='dashed', linewidth=1.5)
        
        # Annotate the lines with standard deviation values
        plt.text(stdev, plt.ylim()[1] * 0.8, f'{std_value}SD= \n \n  {stdev:.2f}', fontsize=10, color='#000000')

    plt.xlabel(f'T2 change (ms)', fontsize=10)
    plt.ylabel('Probability Density', fontsize=10)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.title('T2-Cluster Intensity Threshold', fontsize=12)
    plt.grid(True, alpha=0.5, linewidth=0.25)
    plt.show()


In [ ]:
# Path to your file
cluster_map_all_path = 'ctrl_v2_diffmaps.npy'

# Determine the file extension
file_extension = os.path.splitext(cluster_map_all_path)[1]

# Load the file based on the extension
if file_extension in ['.pkl', '.pickle']:
    with open(cluster_map_all_path, 'rb') as file:
        cluster_map_all = pickle.load(file)
elif file_extension == '.npy':
    cluster_map_all = np.load(cluster_map_all_path)
else:
    raise ValueError("Unsupported file format")
    
# Compyte intensity threshold
compute_intensity_threshold(
    difference_maps_all = cluster_map_all,
    std_values= [2]   # Specifiy all the values at which you want to compute the threshold
)

In [ ]:
mesh_path ='/dataNAS/people/anoopai/KneePipeline/data/10-P/VISIT-5/aclr/results_nsm/femur_mesh_NSM_orig_reg2mean.vtk'



In [ ]:
from scipy.ndimage import label, find_objects

vol_thresh_dict ={}
mesh_path ='/dataNAS/people/anoopai/KneePipeline/data/10-P/VISIT-5/aclr/results_nsm/femur_mesh_NSM_orig_reg2mean.vtk'


# Read the VTK file
femur_mesh = BoneMesh(mesh_path)

# Extract the Diff_T2_mean_int array
diff_T2_mean_int = vtk_to_numpy(femur_mesh.GetPointData().GetArray('Diff_T2_mean_int'))

# Specify the threshold sizes for connected components
size_threshold = 2  # Example threshold, adjust as needed

# Label the connected components (anything not -200)
# Create a mask where values are not -200
mask = diff_T2_mean_int != -200

# Label the connected components
labels, num_features = label(mask)

# Measure the sizes of labeled regions
sizes = np.bincount(labels.ravel())

# Iterate over each component and set values to -200 if size is less than threshold
for i in range(1, num_features + 1):
    if sizes[i] < size_threshold:
        diff_T2_mean_int [labels == i] = -200

print("Filtered Array:", diff_T2_mean_int )

vol_thresh_dict = diff_T2_mean_int 
femur_mesh.point_data[f'Diff_T2_mean_vol'] = vol_thresh_dict 

femur_mesh.save_mesh(mesh_path)

In [ ]:
%load_ext autoreload
%autoreload 2
from fileinput import filename
import SimpleITK as sitk
import pymskt as mskt
import numpy as np
import pandas as pd
import pyvista as pv
import warnings
import torch
import gc
import time
from utils.filter_t2maps import *
from utils.project_t2_data import *

import pymskt.mesh.io as io
from pymskt.image import read_nrrd
from pymskt.mesh.meshTransform import SitkVtkTransformer
from pymskt.mesh.meshTools import ProbeVtkImageDataAlongLine
from pymskt.mesh.meshTools import get_surface_normals, n2l, l2n
from pymskt.mesh.meshes import BoneMesh, CartilageMesh
from pymskt.mesh.utils import is_hit, get_intersect, get_surface_normals, get_obb_surface

path_save = '/dataNAS/people/anoopai/KneePipeline/data/7-P/VISIT-5/aclr/results_nsm'
bone_name = 'femur'
filename_save = 'qdess'

femur_mesh_path = os.path.join(path_save, f'femur_mesh.vtk')
fc_mesh_path  = os.path.join(path_save, f'femur_cart_0_mesh.vtk')
sitk_seg_subregions_path  =  os.path.join(path_save, f'{filename_save}_subregions-labels.nrrd')
sitk_seg_path = os.path.join(path_save, f'{filename_save}_all-labels.nrrd')

# load meshes
mesh_femur = BoneMesh(femur_mesh_path)
mesh_fc = CartilageMesh(fc_mesh_path)
sitk_seg_subregions = sitk.ReadImage(sitk_seg_subregions_path)
sitk_seg = sitk.ReadImage(sitk_seg_path)

# Assign mesh objects
mesh_femur.seg_image = sitk_seg_subregions
cart_labels = [11, 12, 13, 14, 15]
mesh_femur.list_cartilage_labels=cart_labels
mesh_femur.seg_image = sitk_seg
mesh_femur.list_cartilage_meshes = [mesh_fc]

# get the T2 map   
map_path_nrrd= os.path.join(path_save, f'{filename_save}_t2map.nrrd')
map_path_filt_nrrd= os.path.join(path_save, f'{filename_save}_t2map_filt.nrrd')

# filter the T2 map
_ = filter_t2maps(t2_map_path=map_path_nrrd, fwhm= 1, t2_map_save_path=map_path_filt_nrrd)

# project the T2 data onto the femur mesh
data_probe_t2= project_T2_data(mesh_femur, mesh_fc, map_path_nrrd)
mesh_femur.set_scalar('T2_max', data_probe_t2.max_data)
mesh_femur.set_scalar('T2_mean', data_probe_t2.mean_data)
mesh_femur.set_scalar('T2_std', data_probe_t2.std_data)

data_probe_t2_filt= project_T2_data(mesh_femur, mesh_fc, map_path_filt_nrrd)
mesh_femur.set_scalar('T2_max_filt', data_probe_t2_filt.max_data)
mesh_femur.set_scalar('T2_mean_filt', data_probe_t2_filt.mean_data)
mesh_femur.set_scalar('T2_std_filt', data_probe_t2_filt.std_data)

save_path = os.path.join(path_save, f'femur_mesh.vtk')
mesh_femur.save_mesh(save_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
np.unique(data_probe_t2_filt.mean_data)

array([ 0.        , 12.53626974, 13.14231949, ..., 36.18296175,
       36.51046449, 38.04290891])